In [55]:
def build_fs_tree():
    def new_node(dirname):
        return {
            'dir': dirname,
            'subdirs': {},
            'total_size': 0,
            'files': {},
        }

    fs = new_node('/')
    cwd = []

    with open('data.txt', 'r') as terminal_history:
        for line in terminal_history:
            line = line.strip()

            if line.startswith('$ cd '):
                arg = line[len('$ cd '):]
                if arg == '/':
                    cwd.clear()
                    node = fs
                elif arg == '..':
                    cwd.pop()
                    node = fs
                    for f in cwd:
                        node = node['subdirs'].setdefault(f, new_node(f))
                else:
                    cwd.append(arg)
                    # node = node.setdefault(arg, {})
                    node = node['subdirs'].setdefault(arg, new_node(arg))
            elif line=='$ ls':
                pass
            elif line.startswith('dir '):
                arg = line[len('dir '):]
                node['subdirs'].setdefault(arg, new_node(arg))
            else:
                [size, filename] = line.split(' ')
                node['files'].setdefault(filename, size)

                ancestor = fs
                ancestor['total_size'] += int(size)
                for f in cwd:
                    ancestor = ancestor['subdirs'][f]
                    ancestor['total_size'] += int(size)
    return fs

fs = build_fs_tree()

In [56]:
# part one

# To begin, find all of the directories with a total size of at most 100000, then calculate the sum of their total sizes. In the example above, these directories are a and e; the sum of their total sizes is 95437 (94853 + 584). (As in this example, this process can count files more than once!)
# Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?

MAX_SIZE_THRESHOLD = 100000
total_size_below_threshold = 0

def count_sizes_below_threshold(node):
    if node['total_size'] < MAX_SIZE_THRESHOLD:
        return node['total_size'] + sum(count_sizes_below_threshold(x) for x in node['subdirs'].values())
    else:
        return sum(count_sizes_below_threshold(x) for x in node['subdirs'].values())
print(count_sizes_below_threshold(fs))  # assume `/` is above cutoff
# 1543140

1543140


In [86]:
# part two

# The total disk space available to the filesystem is 70000000. To run the update, you need unused space of at least 30000000. You need to find a directory you can delete that will free up enough space to run the update.
# Find the smallest directory that, if deleted, would free up enough space on the filesystem to run the update. What is the total size of that directory?

TOTAL_DISK_SPACE = 70000000
MIN_FREE_SPACE = 30000000

current_free_space = TOTAL_DISK_SPACE - fs['total_size']
target_dir_size = MIN_FREE_SPACE - current_free_space

print(f'target: {target_dir_size}')

def min_size_above_threshold(node, threshold):
    if node['total_size'] > threshold:
        local_min = node['total_size']
        for child_min in (min_size_above_threshold(x, threshold) for x in node['subdirs'].values()):
            if child_min:
                local_min = min(local_min, child_min)
        return local_min

print(f'found: {min_size_above_threshold(fs, target_dir_size)}')  # skip `/`
# target: 1111105
# found: 1117448

target: 1111105
found: 1117448
